## Data Ingestion

In [3]:
### document Structure

from langchain_core.documents import Document

In [4]:
doc = Document(
    page_content="This is the main text content I am using to create RAG ",
    metadata={
        "source": "example.txt",
        "author": "Jitesh",
        "page": 1,
        "date_created": "2025-01-01",
    }
)
doc

Document(metadata={'source': 'example.txt', 'author': 'Jitesh', 'page': 1, 'date_created': '2025-01-01'}, page_content='This is the main text content I am using to create RAG ')

In [5]:
## create a simple text document
import os
os.makedirs("../data/text_files", exist_ok=True)

In [6]:
sample_texts={
    "../data/text_files/python_intro.txt":"""Python Programming Introduction

Python is a high-level, interpreted programming language known for its simplicity and readability.
Created by Guido van Rossum and first released in 1991, Python has become one of the most popular
programming languages in the world.

Key Features:
- Easy to learn and use
- Extensive standard library
- Cross-platform compatibility
- Strong community support

Python is widely used in web development, data science, artificial intelligence, and automation.""",
    
    "../data/text_files/machine_learning.txt": """Machine Learning Basics

Machine learning is a subset of artificial intelligence that enables systems to learn and improve
from experience without being explicitly programmed. It focuses on developing computer programs
that can access data and use it to learn for themselves.

Types of Machine Learning:
1. Supervised Learning: Learning with labeled data
2. Unsupervised Learning: Finding patterns in unlabeled data
3. Reinforcement Learning: Learning through rewards and penalties

Applications include image recognition, speech processing, and recommendation systems
    
    
    """

}

for filepath,content in sample_texts.items():
    with open(filepath,'w',encoding="utf-8") as f:
        f.write(content)

print("✅ Sample text files created!")

✅ Sample text files created!


In [7]:
## Textloader
from langchain.document_loaders import TextLoader

from langchain_community.document_loaders import TextLoader

loader = TextLoader("../data/text_files/python_intro.txt", encoding="utf-8")
document=loader.load()
print(document)

[Document(metadata={'source': '../data/text_files/python_intro.txt'}, page_content='Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popular\nprogramming languages in the world.\n\nKey Features:\n- Easy to learn and use\n- Extensive standard library\n- Cross-platform compatibility\n- Strong community support\n\nPython is widely used in web development, data science, artificial intelligence, and automation.')]


In [8]:
## directory loader
from langchain.document_loaders import DirectoryLoader

dir_loader = DirectoryLoader(
    "../data/text_files", 
    glob="**/*.txt",
    loader_cls=TextLoader, 
    loader_kwargs={"encoding": "utf-8"},
    show_progress=False
)
documents=dir_loader.load()
documents

[Document(metadata={'source': '../data/text_files/python_intro.txt'}, page_content='Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popular\nprogramming languages in the world.\n\nKey Features:\n- Easy to learn and use\n- Extensive standard library\n- Cross-platform compatibility\n- Strong community support\n\nPython is widely used in web development, data science, artificial intelligence, and automation.'),
 Document(metadata={'source': '../data/text_files/machine_learning.txt'}, page_content='Machine Learning Basics\n\nMachine learning is a subset of artificial intelligence that enables systems to learn and improve\nfrom experience without being explicitly programmed. It focuses on developing computer programs\nthat can access data and use it to learn for themselves.\n\nTypes of Machine Learning:\n1. Supervise

In [10]:


from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader

## load all the text files from the directory
dir_loader=DirectoryLoader(
    "../data/pdf_files",
    glob="**/*.pdf",
    loader_cls=PyMuPDFLoader,
    show_progress=False

)

pdf_documents=dir_loader.load()
pdf_documents



[Document(metadata={'producer': 'pdfTeX-1.40.26', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-08-22T13:39:41+00:00', 'source': '../data/pdf_files/Jitesh_resume.pdf', 'file_path': '../data/pdf_files/Jitesh_resume.pdf', 'total_pages': 1, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2025-08-22T13:39:41+00:00', 'trapped': '', 'modDate': 'D:20250822133941Z', 'creationDate': 'D:20250822133941Z', 'page': 0}, page_content='Jitesh Kumar\njitesh.me | jitesh.kumar05official@gmail.com | LinkedIn | GitHub | +91 6397983037\nEducation\nAnurag University\nCGPA: 8.74/10\nBachelor of Technology in Information Technology\n2023 – 2027\nKendriya Vidyalaya Picket, Hyderabad\nPercentage: 81%\nCBSE 12th\n2021 – 2023\nProjects\nGo To Buddy – AI-Powered Desktop Assistant | Link\n• Engineered a multi-threaded desktop assistant using Python & PyQt6 to ensure low-latency, real-time\nresponsiveness for voice commands.\n• Arch: PyQt6 UI →Multi-threaded Backen

In [11]:
type(pdf_documents[0])

langchain_core.documents.base.Document

## Embedding and VectorStoreDB

In [12]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List,Dict, Any,Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [15]:
class EmbeddingManager:
    """Handles Document Embedding using SentenceTransformers and stores in ChromaDB"""
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initializes the EmbeddingManager.
        Args:
            model_name: Hugging Face model name for sentence embeddings.
        """
        self.model_name = model_name
        self.model=None
        self._load_model()

    def _load_model(self):
        """Loads the SentenceTransformer model."""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model: {e}")
            raise
    
    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generates embeddings for a list of texts.
        Args:
            texts: List of text strings to embed.
        Returns:
            Numpy array of embeddings.
        """
        if not self.model:
            raise ValueError("Model not loaded.")
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, convert_to_numpy=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings
    
    

## initialize Embedding Manager
embedding_manager=EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2


/home/jitesh/Desktop/Programing/Python/RAG_Learn_Bas1/.venv/lib/python3.11/site-packages/torch/cuda/__init__.py:182: UserWarning: CUDA initialization: CUDA driver initialization failed, you might not have a CUDA gpu. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


Model loaded successfully. embedding dimension: 384
